# Google BigQuery API
Scrape gitHub repo's content and commit records using BigQuery API
- need to apply for an account 
- set API request to connect to jupyter notebook

## Connect to BigQuery

In [1]:
from google.cloud import bigquery

In [2]:
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] ='/sfs/lustre/bahamut/scratch/yt9mh/RA-github/data/miles.json' # key json file


In [3]:
client = bigquery.Client()

## Get commit
Get commit by repo name

In [ ]:
from google.cloud import bigquery
import pandas as pd

client = bigquery.Client()

query = """

SELECT
    commit AS commit,
    TIMESTAMP_SECONDS(committer.time_sec) AS commit_time,
    message AS message,
    repo AS repo,
    diff.new_sha1

FROM `bigquery-public-data.github_repos.commits`, UNNEST(repo_name) repo, UNNEST(difference) diff
WHERE repo IN (
    'IndonesiaX/edx-platform'

)            
    """

query_job = client.query(query)

iterator = query_job.result(timeout=30)
rows = list(iterator)
commit = pd.DataFrame(data=[list(x.values()) for x in rows], columns=list(rows[0].keys()))

# Look at the first 10 headlines
# commit.to_csv('commit/new_scrape/py_ml_commit1.csv', index = False) # save data to csv file
commit.head(10)

In [8]:
commit.groupby(['repo']).agg({'commit':lambda x: x.nunique()}) # see commit total count

,commit
repo,
18F/caddy-boshrelease,13
18F/federalist-build-js,48
18F/microsite-18f,19
18F/sqlalchemy-jsonapi,39
18F/wg-onboarding,5
...,...
zulip/zulip-js,138
zumba/swivel-cake,50
zumba/zlogd,23


## Get content
Get repo name and content based on company

In [6]:
from google.cloud import bigquery
import pandas as pd

client = bigquery.Client()

query = """
   SELECT c.id, c.content, path, repo_name
  
FROM `bigquery-public-data.github_repos.files` f
JOIN `bigquery-public-data.github_repos.contents` c
ON f.id = c.id
WHERE repo_name LIKE 'brilliantorg/%' or
    repo_name LIKE 'quora/%' or
    repo_name LIKE 'salesforce/%' or
    repo_name LIKE 'target/%' or
    repo_name LIKE 'netapp/%' or
    repo_name LIKE 'nvidia/%' or
    repo_name LIKE 'instagram/%' or
    repo_name LIKE 'snapchat/%' or
    repo_name LIKE 'samsung/%' or
    repo_name LIKE 'shopify/%' or
    repo_name LIKE 'amzn/%'

            """


query_job = client.query(query)

iterator = query_job.result(timeout=30)
rows = list(iterator)

commit = pd.DataFrame(data=[list(x.values()) for x in rows], columns=list(rows[0].keys()))

# Look at the first 10 headlines
# commit.to_csv('not_py_ml_commit/commit5.csv', index = False)
commit.head(10) 

,id,content,path,repo_name
0,03e8e690d5c53108349a6899ddc4272697f7f873,# Node.js for Alexa Connected Home Skills\n\n#...,README.md,amzn/alexa-coho
1,8dada3edaf50dbc082c9a125058f25def75e625a,Apache Licens...,LICENSE.txt,amzn/alexa-coho
2,b7d179b3709f803050659844154d3ae08af86484,Alexa Connected Home Skill\nCopyright [2015]-[...,NOTICE.txt,amzn/alexa-coho
3,5ce5d02fb57fbfca507ae68adc90702cee049ec3,# Project Status: Inactive\n\nThis project wil...,README.md,target/dse-cookbook
4,79b61c9659fd61f4dbfccf36e1f64819af2648a1,# mac\n.DS_Store\n\n# Byte-compiled / optimize...,.gitignore,target/webbreaker
5,bcf335208f97560a63b7398eb8b3e239894b28af,"/**\n Copyright 2014-2015 Amazon.com, Inc. ...",LightingAPI/SampleVirtualDevice.js,amzn/alexa-coho
6,eb9b6f43f2afaba44277b84d2d86baad49c960a5,# Project Status: Inactive\n\nThis project wil...,README.md,target/webbreaker
7,90d01f74b7056682aff6c4554ed54da4ecab8922,# Node.js for Alexa Connected Home Skills\n\n#...,LightingAPI/README.md,amzn/alexa-coho
8,6755997726614da1172cf894396ac9dc2c6850c7,# Project Status: Inactive\n\nThis project wil...,README.md,target/lita-hipchat-admin
9,2d582029c635f780e111b70d5a784bcf2d32ac0f,"/*\n * Copyright 2016 Quora, Inc.\n *\n * Lice...",qmf/metrics/MetricsManager.h,quora/qmf
